In [1]:
pip install scikit-bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 28.8 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 20.1 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-bio: filename=scikit_bio-0.6.3-cp311-cp311-macosx_11_0_arm64.whl size=4805318 sha256=2a14bb106a77fff8aa9c5cde498c4230f193f5678090bb8b3c89d2fe015cc2d9
  Stored in directory: /Users/sb/Library/Caches/pip/wheels/c9/11/a2/86492071506a487bcba7015b710a9e89d66ca69845512fb86c
  Created wheel for biom-format: filename=biom_format-2.1.16-cp311-cp311-macosx_11_0_arm64.whl size=11780615 sha256=728fd4715ad211e426efcb35c002e2e14db6db256e3b1ab637fac64a3b17deba
  Stored in directory: /Users/sb/Library/Caches/pip/wheels/a5/6b/58/a879e8

In [2]:
pip install biopython

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from Bio import Phylo

##############################################################################
# 1. LOAD / PREPARE YOUR DATA
##############################################################################

# ----------------------------------------------------------------------------
# Assume you already have a DataFrame called df with shape: (num_samples, num_otus).
# Rows = samples, Columns = OTUs, and the values are abundances (counts, relative abundances, etc.).
# If you have extra metadata columns, remove or separate them so that only OTU columns remain.

# Example (dummy) df creation if you need to simulate:
# Comment this out if you already have df
# np.random.seed(42)
# df = pd.DataFrame(
#     np.random.randint(0, 100, (10, 5)),  # 10 samples, 5 OTUs
#     columns=["OTU_1", "OTU_2", "OTU_3", "OTU_4", "OTU_5"]
# )

print("DataFrame shape:", df.shape)
print("DataFrame head:\n", df.head())

##############################################################################
# 2. LOAD / PARSE YOUR PHYLOGENETIC TREE
##############################################################################

# ----------------------------------------------------------------------------
# Suppose you have a phylogenetic tree in Newick format saved in "tree.nwk".
# Make sure the tip labels in the tree correspond to your OTU names in df.columns.

# tree = Phylo.read("tree.nwk", "newick")

# ----------------------------------------------------------------------------
# For demonstration, we’ll create a simple example tree with Biopython:
# You can comment this out and load your actual tree above.

from io import StringIO
example_tree_newick = "(OTU_4:0.1,(OTU_2:0.2,(OTU_1:0.1,OTU_3:0.3):0.2):0.3,OTU_5:0.4);"
tree = Phylo.read(StringIO(example_tree_newick), "newick")

print("\nPhylogenetic Tree Tips (OTUs) in order:")
Phylo.draw_ascii(tree)

##############################################################################
# 3. DETERMINE A PHYLOGENETIC ORDER OF OTUs
##############################################################################

# ----------------------------------------------------------------------------
# Extract tip names in a left-to-right traversal (or any consistent traversal).
# The tip order will define the rearrangement of your columns.

def get_tip_labels(tree):
    """Return tip labels (OTU names) from the tree in a consistent traversal order."""
    tip_labels = []
    for tip in tree.get_terminals():
        tip_labels.append(tip.name)
    return tip_labels

phylo_otus = get_tip_labels(tree)
print("\nPhylogenetic OTU order:", phylo_otus)

# ----------------------------------------------------------------------------
# Reorder the columns of df based on the tree’s tip labels
# Only keep columns that actually appear in df (in case your tree has extra tips or vice versa).
phylo_otus_in_df = [otu for otu in phylo_otus if otu in df.columns]
df_ordered = df[phylo_otus_in_df]

print("\nDataFrame columns after reordering by phylogenetic order:")
print(df_ordered.columns)

##############################################################################
# 4. CONVERT THE ORDERED OTU ABUNDANCE ROW INTO A 2D "IMAGE"
##############################################################################

# ----------------------------------------------------------------------------
# Many approaches exist for turning 1D vectors into 2D grids:
#   - Simple: Reshape the vector (num_otus,) into something like sqrt(num_otus) x sqrt(num_otus)
#   - More complex: Use pairwise distances or the actual tree structure to embed in 2D
#
# Here, we'll do a simplistic approach: if you have M OTUs, we make an L x L image
# where L = int(ceil(sqrt(M))). We fill in row-major order, or zero-pad if needed.

import math

def vector_to_square_image(vector):
    """Reshape a 1D vector into the smallest square (zero-padded if necessary)."""
    m = len(vector)
    size = int(math.ceil(math.sqrt(m)))
    image = np.zeros((size, size), dtype=float)
    # Fill in row-major order
    image.ravel()[:m] = vector
    return image

# ----------------------------------------------------------------------------
# We'll transform each sample's phylo-ordered OTU abundances into a 2D image.
# If we have N samples, we get N images, each shape = (size, size).

def df_to_image_array(df_phylo):
    """Convert each row of df_phylo into a 2D image, return a 3D array: (num_samples, size, size)."""
    images = []
    for idx, row in df_phylo.iterrows():
        vec = row.values
        img_2d = vector_to_square_image(vec)
        images.append(img_2d)
    return np.array(images)

images_array = df_to_image_array(df_ordered)
print("\nImages array shape:", images_array.shape)

# If images_array has shape (num_samples, height, width), for a CNN we often need a channel dimension.
# e.g. shape -> (num_samples, height, width, 1)
images_array = np.expand_dims(images_array, axis=-1)
print("Images array shape (with channel):", images_array.shape)

##############################################################################
# 5. PREPARE LABELS (Y) FOR CLASSIFICATION OR REGRESSION
##############################################################################

# ----------------------------------------------------------------------------
# Typically, you have some phenotype or class label per sample. For example:
#   - a disease status (case/control)
#   - or a numeric trait
#
# We'll create dummy labels for demonstration. Suppose we do a binary classification.

num_samples = images_array.shape[0]
# Dummy: label half the samples as 0, half as 1
dummy_labels = np.array([0 if i < num_samples/2 else 1 for i in range(num_samples)])

# If you have real labels in e.g. df_labels, you’d do:
# dummy_labels = df_labels.values

print("\nLabels shape:", dummy_labels.shape)

##############################################################################
# 6. SPLIT INTO TRAIN/TEST (OR TRAIN/VAL/TEST)
##############################################################################

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    images_array,
    dummy_labels,
    test_size=0.2,
    random_state=42
)

print("\nTrain set X shape:", X_train.shape, "y shape:", y_train.shape)
print("Test set X shape:", X_test.shape, "y shape:", y_test.shape)

##############################################################################
# 7. BUILD A SIMPLE CNN MODEL
##############################################################################

def create_cnn_model(input_shape):
    """
    Create a simple CNN model using Keras/TF.
    input_shape = (height, width, channels)
    """
    model = Sequential()
    model.add(Conv2D(filters=8, kernel_size=(3,3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(filters=16, kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # for binary classification
    
    # Compile
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

model = create_cnn_model(input_shape=X_train.shape[1:])
model.summary()

##############################################################################
# 8. TRAIN THE CNN
##############################################################################

history = model.fit(
    X_train,
    y_train,
    epochs=10,             # Increase epochs for real data
    batch_size=2,          # Adjust batch size to your data size
    validation_split=0.2,  # For demonstration, splits off part of training set for validation
    verbose=1
)

##############################################################################
# 9. EVALUATE ON TEST SET
##############################################################################

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")

##############################################################################
# 10. PREDICT / USE THE MODEL
##############################################################################

predictions = model.predict(X_test)
predicted_classes = (predictions > 0.5).astype(int).flatten()

print("\nPredictions (raw):", predictions.flatten())
print("Predicted classes:", predicted_classes)
print("True classes:", y_test)
